In [1]:
##############################
#Training loop for NNs       #
#Maintainer: Christopher Chan#
#Version: 0.0.5              #
#Date: 2022-02-15            #
##############################

import os
import sys
import torch
import pathlib
import time
import numpy as np
import torch.nn as nn
import segmentation_models_pytorch as smp
from torch import optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torch.utils.tensorboard import SummaryWriter
from Networks import Five_UNet
from dataloader import BuildingDataset

device = (torch.device("cuda") if torch.cuda.is_available()
          else torch.device("cpu"))

print(f"Training on device {device}.")

Training on device cpu.


### Train Val Test split

In [10]:
td_KBY = os.path.abspath("/home/chris/Dropbox/HOTOSM/SAMPLE/td_KBY")
td_DZK = os.path.abspath("/home/chris/Dropbox/HOTOSM/SAMPLE/td_DZK")
td_DZKN = os.path.abspath("/home/chris/Dropbox/HOTOSM/SAMPLE/td_DZKN")

#td_KBY = os.path.abspath("/home/mnt/HOTOSM_data/Kakuma/Kalobeyei/td_KBY")
#td_DZK = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka/td_DZK")
#td_DZKN = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka_N/td_DZKN")
#
# Below is a set of relatively complex functions which:
# Perform the train, val, test split at a rounded ratio of 62%, 27%, and 10% based on each sets of imagery
# This will be followed by first pseudo changing the name of _LBL_ to _IMG_ to match the split imagery
# Lastly, once the correct LBL files are matched, 

def tvt_split(td):
    
    img_ls = []
    
    for root, dirs, filename in os.walk(os.path.join(td, "IMG")):
        for i in filename:
            if i.endswith(".png"):
                img_ls.append(root + "/" + i)
        
        img_ls = BuildingDataset(img_ls, _)
        
        train_IMG, val_IMG, test_IMG = random_split(img_ls.png_dir, [int(round(0.6 * len(img_ls.png_dir))),
                                                                     int(round(0.3 * len(img_ls.png_dir))),
                                                                     int(round(0.1 * len(img_ls.png_dir)))])
        
        return train_IMG, val_IMG, test_IMG

DZK_train, DZK_val, DZK_test = tvt_split(td_DZK)
KBY_train, KBY_val, KBY_test = tvt_split(td_KBY)
DZKN_train, DZKN_val, DZKN_test = tvt_split(td_DZKN)

def match_LBL(td, imgs):
    
    lbl_ls = []
    img_ls = []
    match_ls = []
    
    imgs = list(imgs)
    
    for root, dirs, filename in os.walk(os.path.join(td, "LBL")):
        for j in filename:
            if j.endswith(".png"):
                ps_name = j.rsplit("_LBL_")[0] + "_IMG_" + j.rsplit("_LBL_")[1] # Parse the string, PSEUDO-CHANGE _LBL_ to _IMG_
                lbl_ls.append(ps_name)
    
    for k in imgs:
        names = os.path.basename(k)
        img_ls.append(names)
        
    def common(a, b):
        a_set = set(a)
        b_set = set(b)
        if (a_set & b_set):
            return (a_set & b_set)
        else:
            print("No common elements")
            
            
    match_ls = common(img_ls, lbl_ls)
        
    match_ls = [(root + "/" + n.replace("_IMG_", "_LBL_")) for n in match_ls] # Change the _IMG_ back to _LBL_
    
    print("For the selected dataset of {0}, There are: {1} images, {2} labels, and {3} matching image/label pairs.".format(os.path.basename(td), len(img_ls), len(lbl_ls), len(match_ls)))
    
    return match_ls


#########################################
# Assign matched LBL to new LBL datasets#
#########################################

DZKLBL_Train = match_LBL(td_DZK, DZK_train)
DZKLBL_Val = match_LBL(td_DZK, DZK_val)
DZKLBL_Test = match_LBL(td_DZK, DZK_test)
DZKNLBL_Train = match_LBL(td_DZKN, DZKN_train)
DZKNLBL_Val = match_LBL(td_DZKN, DZKN_val)
DZKNLBL_Test = match_LBL(td_DZKN, DZKN_test)
KBYLBL_Train = match_LBL(td_KBY, KBY_train)
KBYLBL_Val = match_LBL(td_KBY, KBY_val)
KBYLBL_Test = match_LBL(td_KBY, KBY_test)

DZKLBL_Train = BuildingDataset(_, [DZKLBL_Train])
DZKLBL_Val = BuildingDataset(_, [DZKLBL_Val])
DZKLBL_Test = BuildingDataset(_, [DZKLBL_Test])
DZKNLBL_Train = BuildingDataset(_, [DZKNLBL_Train])
DZKNLBL_Val = BuildingDataset(_, [DZKNLBL_Val])
DZKNLBL_Test = BuildingDataset(_, [DZKNLBL_Test])
KBYLBL_Train = BuildingDataset(_, [KBYLBL_Train])
KBYLBL_Val = BuildingDataset(_, [KBYLBL_Val])
KBYLBL_Test = BuildingDataset(_, [KBYLBL_Test])


Train = BuildingDataset(png_dir = (DZK_train + KBY_train + DZKN_train),
                        lbl_dir = (DZKLBL_Train + KBYLBL_Train + DZKNLBL_Train))

Val = BuildingDataset(png_dir = (DZK_val + KBY_val + DZKN_val),
                      lbl_dir = (DZKLBL_Val + KBYLBL_Val + DZKNLBL_Val))

Test = BuildingDataset(png_dir = (DZK_test + KBY_test + DZKN_test),
                       lbl_dir = (DZKLBL_Test + KBYLBL_Test + DZKNLBL_Test))

print(len(Train.png_dir) == len(Train.lbl_dir))
print(type(Train.png_dir), type(Train.lbl_dir))

print("Total images and labels pair in DataLoader: {0}".format(len(Train.png_dir) + len(Val.png_dir) + len(Test.png_dir)))

print("Concatenated TRAINING images and labels pair: {0} :".format(len(Train.png_dir)))
for x, y in zip(Train.png_dir, Train.lbl_dir):    
    print(f"Image: {x}", f"Label: {y}")

print("Concatenated VALIDATION images and labels pair: {0} :".format(len(Val.png_dir)))
for x, y in zip(Val.png_dir, Val.lbl_dir):
    print(f"Image: {x}", f"Label: {y}")

print("Concatenated TESTING images: {0} and labels pair: {0} :".format(len(Test.png_dir)))
for x, y in zip(Test.png_dir, Test.lbl_dir):
    print(f"Image: {x}", f"Label: {y}")


For the selected dataset of td_DZK, There are: 6 images, 10 labels, and 6 matching image/label pairs.
For the selected dataset of td_DZK, There are: 3 images, 10 labels, and 3 matching image/label pairs.
For the selected dataset of td_DZK, There are: 1 images, 10 labels, and 1 matching image/label pairs.
For the selected dataset of td_DZKN, There are: 6 images, 10 labels, and 6 matching image/label pairs.
For the selected dataset of td_DZKN, There are: 3 images, 10 labels, and 3 matching image/label pairs.
For the selected dataset of td_DZKN, There are: 1 images, 10 labels, and 1 matching image/label pairs.
For the selected dataset of td_KBY, There are: 6 images, 10 labels, and 6 matching image/label pairs.
For the selected dataset of td_KBY, There are: 3 images, 10 labels, and 3 matching image/label pairs.
For the selected dataset of td_KBY, There are: 1 images, 10 labels, and 1 matching image/label pairs.


TypeError: 'tuple' object cannot be interpreted as an integer

In [29]:
train_loader = DataLoader(Train, batch_size = 1, shuffle = False)
val_loader = DataLoader(Val, batch_size = 1, shuffle = False)
print(train_loader)

In [9]:

(KBY_train.dataset.sort)

print(KBY_train.)

In [25]:
Five_UNet = Five_UNet()

In [33]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device = device)
            labels = labels.to(device = device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print("{} Epoch {}, Training loss {}".format(
                datetime.datetime.now(), epoch,
                loss_train/len(train_loader)))

train_loader = DataLoader(Train, batch_size = 1, shuffle = False)
val_loader = DataLoader(Val, batch_size = 1, shuffle = False)

model = Five_UNet
model.to(device = device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim = 1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name, correct/total))

In [38]:
training_loop(n_epochs = 100,
              optimizer = optimizer,
              model = model,
              loss_fn = loss_fn,
              train_loader = train_loader)

TypeError: 'tuple' object cannot be interpreted as an integer